# Воспроизведение получения данных для исследования ЦПУР «Качество проведения оценки регулирующего воздействия в России: что показывает сплошной анализ текстовых данных?»

Этот ноутбук – единая точка запуска всех скриптов, которые __собирают и обрабатывают данные__ с regulation.gov.ru и sozd.duma.gov.ru для дальнейшего использования в анализе.

Ноутбук состоит из трех частей: __инициализации__, __получения датасета__, аналогичного датасету, [опубликованному на сайте ИНИД]('https://data-in.ru/data-catalog/datasets/177/'), и __нормализации__ заполнений граф сводных отчетов на отсутствующие, мусорные и содержательные.

Получение датасета реализовано двумя способами: полной перегенерацией датасета, воспроизводящей работу аналитиков ЦПУР, и скачиванием датасета с сайта ИНИД.

<div class="alert alert-block alert-info">
<b>Важно:</b> Сбор исходного датасета, размещённого на ИНИД, занимает <b>длительное время</b>: требуется обойти несколько тысяч страниц электронных порталов, а затем конвертировать, распознавать и парсить длинные документы. Поэтому рекомендуем пропустить эту часть и вместо неё переходить к части «Вариант 2: скачать датасеты с ИНИД»
</div>

## Оглавление:
* [Инициализация](#test)
* [Вариант 1: скачать датасеты с ИНИД](#data-in-download)
* [Вариант 2: полностью перегенерировать данные](#full-data-gen)
* [Нормализация](#norm)

## Установка зависимостей и инициализация <a class="anchor" id="test"></a>

У этого скрипта есть несколько зависимостей: как стандартных, которые можно поставить через `pip`, так и другой собственный репозиторий ЦПУР. При запуске блока кода ниже всё поставится само.

Если в какой-то момент выполнения кода вы увидите ошибку, что вам не хватает некого питоновского пакета, вы можете дописать его название в новую строчку файла `pip-requirements.txt` и снова запустить этот блок.

In [ ]:
# Установим стандартные библиотеки через pip
!pip install --quiet -r pip-requirements.txt

# Установим библиотеку, разработанную ЦПУР для анализа отчётов
!./install_report_parser.sh

Перед тем, как выполнять следующий этап, проверьте, что вы заполнили всё, что хотели, в конфигурационном файле `config.json` (подробнее см. `README.md`)

In [ ]:
# Загрузим конфигурационный файл
import json

config = json.load(open('config.json', 'r'))
workdir = config['working_directory']

In [ ]:
# Добавим в PYTHONPATH ссылки на вспомогательные скрипты

import os
import sys

relative_lib_paths = ['utils/report_parser/',
                      'utils/cleanup_utils/',
                      'utils',
                      'scraping',
                      'gathering',
                      'normalization'
                     ]

absolute_lib_paths = [os.path.abspath(x) for x in relative_lib_paths]
for path in absolute_lib_paths:
    sys.path.insert(0, path)

from utils.report_parser.report import Report
# Протестируем, что Report загрузился
report_example = 'utils/report_parser/examples/report.html'
Report(report_example)

from utils.db_helper import DBHelper
# Протестируем, что DBHelper загрузился
_ = DBHelper(config['database'])

In [ ]:
# Создадим рабочую папку, если её ещё нет

if not os.path.isdir(workdir):
    os.mkdir(workdir)

In [ ]:
# Загрузим другие полезные вещи

import pandas as pd
import pickle
import json

## Вариант 1: скачать датасеты с ИНИД <a class="anchor" id="data-in-download"></a>

В следующем блоке данные загрузятся с сайта каталога.

In [48]:
# Скачаем и сложим датасеты в workdir/datasets

import zipfile
import urllib.request

data_dir = os.path.join(workdir, 'datasets/')
if not os.path.isdir(data_dir):
    os.mkdir(data_dir)

url = '''
https://ds1.data-in.ru/Aggregated_datasets/Normotvorcheskii_process_v_RF_177_27.09.21/Normotvorcheskii_process_v_RF_177_27.09.21.zip'''
with urllib.request.urlopen(url) as response, open(archive, 'wb') as out_file:
    data = response.read() 
    out_file.write(data)
    
with zipfile.ZipFile(archive, 'r') as zip_ref:
    zip_ref.extractall(data_dir)

## Вариант 2: полностью перегенерировать данные <a class="anchor" id="full-data-gen"></a>

В этом варианте получения данных все данные будут сгенерированы из первоисточников с нуля. 

<div class="alert alert-block alert-info">
<b>Важно:</b> Если вы выполнили предыдущий пункт, этот проходить необзятельно.
</div>

[Перейти к следующей части](#norm)

### Скачаем данные проектов на ОРВ

#### Скачаем метаданные regulation.gov.ru

In [20]:
!scraping/regulation_parser.py -c config.json

Started adding projects to queue...
Queue submitted.
Scraper finished.
Dumper is empty.
Executors finished
Tasks empty: True
No tasks left
No results left
0:05:35.342103


#### Cкачаем файлы сводных отчетов и текстов НПА

In [25]:
!scraping/download_files.py -c config.json -i orv

Task done
Task done
Task done
Task done
Task done
Task done
Task done
Task done
Task done
Task done
No tasks left for this worker
executors finished
no tasks left
[]


### Скачаем данные законопроектов, внесенных в Госдуму

#### Сбор метаданных по АПИ

In [ ]:
!scraping/duma_parser.py -c config.json

#### Файлы законопроектов

In [32]:
!scraping/download_files.py -c config.json -i duma

Task done
No tasks left for this worker
executors finished
no tasks left
[]


### Соберём сводные отчёты из файлов

На этом этапе мы с помощью библиотеки [report-parser]('https://github.com/CAG-ru/report_parser') преобразуем содержимое файлов, которые пока хранятся в формате отдельных файлов, в коллекцию питоновских объектов типа `Report` – то есть объектов, содержащих тексты и таблицы в формате `pandas.DataFrame`.

Это долгий процесс, можете оставить скрипт работать, и пойти выпить самовар чаю – результаты на всякий случай сохранятся на диск.

In [13]:
files_df = pd.DataFrame(columns=['regulation_project_id',
                                'relative_path'])

for degree in ['Низкая', 'Высокая', 'Средняя']:
    files_dir = os.path.join(workdir, 
                            'scraping/main_ria_report') + '_' + degree   
    pickled_path = os.path.join(workdir, 
                            'scraping/preparsed_reports') + '_' + degree
    
    reports = []
    
    for fn in os.listdir(files_dir):
        report_path = os.path.join(files_dir, fn)
        project_id = int(os.path.splitext(fn)[0])
        files_df = files_df.append(pd.Series({
            'regulation_project_id': project_id,
            'relative_path' : report_path
            }), ignore_index=True)
        try:
            report= Report(report_path)
            report.parse()
            reports.append((project_id, report))
        except Exception as e:
            print(fn, e)
            
    with open(pickled_path, 'wb') as f:
        pickle.dump(reports, f)

15096.docx File is not a zip file
20367.pdf Расширение pdf не поддерживается


### Структурируем данные сводных отчетов

На этом этапе проведём парсинг объектов типа `Report` в плоские таблицы: 1 основную и 15 вспомогательных.
В основной таблице одной строчке соответствует сводный отчёт одного проекта, а колонки соответствуют графам отчёта, которые не предполагают множественных ответов.

В случаях, когда графа отчёта может быть заполнена целым списком значений (например, «Цели предполагаемого регулирования»), для неё создаётся отдельная таблица с `one-to-many` соответствием: например, одному и тому же отчёту могут соответствовать несколько строк с разными целями регулирования.

In [ ]:
# Заберем метаинформацию, она поможет нам с парсингом

db = DBHelper(config['database'])
query = '''
    SELECT regulation_project_id, regulatory_impact
    FROM public.project
'''
task = pd.DataFrame(db.select_statement(query),
                   columns=['regulation_project_id', 
                            'impact_degree'])


In [ ]:
# Подцепим назад записанные на диск Report'ы

reports = []
for degree in ['Низкая', 'Высокая', 'Средняя']:
    pickled_dir = os.path.join(workdir, 
                               'scraping/preparsed_reports') + '_' + degree 
    reports.extend(pickle.load(open(pickled_dir, 'rb')))

In [ ]:
# Загрузим пустые датафреймы, куда будем складывать результаты

with open("gathering/df_primers", "rb") as f:
    df_primers = pickle.load(f)
main_df = df_primers["main_df"]
otm_tables = df_primers["otm_tables"]

In [ ]:
# Запустим парсеры, которые структурируют информацию из отчетов

from gather_to_dataframe_utils import add_report_to_df, fill_blanks

bad_reports = []
for project_id, report in reports:
    main_df = add_report_to_df(project_id, report, main_df, otm_tables, bad_reports)

In [ ]:
# Удалим лишние колонки – они возникают, когда кто-то случайно создает лишний заголовок в отчете
main_df = main_df.iloc[:, :73]

# Непустой 'header: id' – хороший предиктор адекватности отчета, 
# т.к. он заполняется автоматически и должен быть всегда
main_df = main_df[~main_df['header: id'].isnull()]

# Добавим нужные колонки и удалим ненужные
main_df.drop(['header: id', 'header: regulation_project_id',
              'header: start', 'header: end'], 
             inplace=True, axis=1)
main_df = main_df.merge(task, on='regulation_project_id', how='inner')

# Пометим специальным значением non-applicable графы, которые и не должны были быть заполнены
main_df = main_df.apply(fill_blanks, axis=1)
# Степень регулирующего воздействия больше не нужна
main_df.drop(['impact_degree'], 
             inplace=True, axis=1)

In [ ]:
from orv_cleanup_utils import *

for column in main_df.columns:
    main_df[column] = main_df[column].apply(remove_standard_form)

In [ ]:
# Запишем данные в ту же директорию, как если бы мы их скачали

parsed_dir = os.path.join(workdir, 'datasets/')

os.makedirs(parsed_dir, exist_ok=True)

main_df.to_csv(os.path.join(parsed_dir, 'ria_reports_main.csv'), 
             index=False)

for otm_name, otm_df in otm_tables.items():
    if otm_name == 'business':
        otm_df[0].to_csv(os.path.join(parsed_dir,
                'ria_reports_business_sizes_as_is.csv'), 
                index=False)
        otm_df[1].to_csv(os.path.join(parsed_dir,
                'ria_reports_business_profit_loss.csv'), 
                index=False)
        otm_df[1].to_csv(os.path.join(parsed_dir,
            'ria_reports_business_sizes_to_be.csv'), 
             index=False)
        continue
        
    otm_df.to_csv(os.path.join(parsed_dir, 'ria_reports_' \
                               + otm_name + '.csv'), 
                               index=False)

## Нормализация <a class="anchor" id="norm"></a>

Вы дошли до нормализации – вы восхитительны!

Во время нормализации происходит разметка на пустые, несодержательные и содержательные заполнения.

### Итоговая разметка мусорных и отсутствующих заполнений

1. Разметка пустых, несодежрательных заполнений происходила следующим образом. Типичные кандидаты в «мусор», как правило, представляют собой наиболее частые и/или короткие заполнения в каждой графе. Для их отлова нам понадобятся `most_freq` и `shortest` из `orv_cleanup_utils`.
2. После применения этих функций к каждой из граф было взяты по топ-`50` самых частых и самых коротких заполнений. Таблицы с ними и были направлены экспертам.
3. Эксперты разметили каждое заполнение графы `0` или `1` - в зависимости от содержательности заполнения.
4. Заполнения, отмеченные `0`, были взяты в качестве негативной оценки заполненности графы.
5. Все графы были размечены по своим "словарикам" правильных и неправильных ответов.
6. Документ `junk_by_field.json` содержит негативные заполнения по каждому из полей формы, а `mapping_fields.json` позволяет по короткому наименованию поля получить расширенное - то, которое использовалось в отчёте.
7. В результате обработки были созданы таблицы, каждая из которых содержит оригинальное заполнение, а также проставленную оценку в соседнем столбце.

In [ ]:
# Загрузим датасеты

data_dir = os.path.join(workdir, 'datasets/')
main_df = pd.read_csv(data_dir + 'ria_reports_main.csv', encoding='utf-8', delimiter=';')

otm_tables_fnames = sorted([fn for fn in os.listdir(data_dir) if not (fn.startswith('main') or fn.startswith('.'))])
otm_tables = {fn[:-4]: pd.read_csv(data_dir + fn, encoding='utf-8', delimiter=';') for fn in otm_tables_fnames}
otm_tables.keys()

In [ ]:
import json
from normalization.orv_cleanup_utils import fill_df_info


with open('normalization/junk/mapping_fields.json', 'r') as fp:
    all_fields = json.load(fp)
    

# junk_main_df = fill_df_info(all_fields, main_df, 'main')
# junk_main_df.to_excel('normalization/junk/junk_main_df.xlsx', header=True)

for _ in otm_tables.keys():
    short_name = _.replace('ria_reports_', '')
    if short_name in all_fields.keys():
        df = otm_tables.get(_)
        print(_)
        result = fill_df_info(all_fields, df, short_name)
        result.to_excel(f'normalization/junk/junk_{_}_df.xlsx', header=True)